# Import

In [1]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf            
import sklearn.model_selection    
import keras.preprocessing.image  
import datetime                   
import skimage.io           
import sys                        
import tqdm                       
import seaborn as sns             
import matplotlib.cm as cm        
%matplotlib inline       

Using TensorFlow backend.


# Define parameters

In [2]:
# Global constants.
IMG_WIDTH = 256     
IMG_HEIGHT = 256      
IMG_CHANNELS = 3      
CW_DIR = os.getcwd()  
TRAIN_DIR = os.path.join(CW_DIR, 'input', 'stage1_train')
TEST_DIR = os.path.join(CW_DIR, 'input', 'stage1_test')
IMG_TYPE = '.png'         
IMG_DIR_NAME = 'images'   
MASK_DIR_NAME = 'masks'   
LOGS_DIR_NAME = 'logs'    
SAVES_DIR_NAME = 'saves'  
SEED = 42                 

# Global variables.
min_object_size = 1       # Minimal nucleous size in pixels
x_train = []
y_train = []
x_test = []
y_test_pred_proba = {}
y_test_pred = {}

# Generate lookup table for quick reference

In [ ]:
id_list = os.listdir(TRAIN_DIR)
id_list.remove('.DS_Store')

lookup_table = pd.DataFrame()
tmp = []
for i in np.arange(0, len(id_list)):
    img_name_id = id_list[i]
    file = "input/stage1_train/{}/images/{}.png".format(id_list[i], id_list[i])
    masks_path = "input/stage1_train/{}/masks/*.png".format(id_list[i])
    image = cv2.imread(file)
    height, width, channels = image.shape
    tmp.append(['{}'.format(img_name_id), width, height])
lookup_table = pd.DataFrame(tmp, columns = ['img_id', 'img_width', 'img_height'])


In [ ]:
type_names = {0: "nissl-stained", 
              1: "fluorescent", 
              2: "bright-field"}

def get_violet_num(img):
    violet_num = 0
    h, w = img.shape[:2]
    for y in range(h):
        for x in range(w):
            if img[y, x][0] > img[y, x][1] and img[y, x][2] > img[y, x][1]:
                violet_num += 1

    return violet_num

def get_microscopy_type(img):
    violet_num = get_violet_num(img)
    if violet_num > 0:
        return 0
    mean_int = img.mean()
    if mean_int > 110:
        return 2
    return 1

types_list=[]
for i in lookup_table['img_id']:
    file_path = "input/stage1_train/{}/images/{}.png".format(i, i)
    image = skimage.io.imread(file_path)
    types_list.append(type_names[get_microscopy_type(image)])
lookup_table['type'] = types_list

In [ ]:
df_shape = pd.DataFrame([[x] for x in zip(lookup_table['img_width'], 
                                          lookup_table['img_height'], lookup_table['type'])])
df_shape[0].value_counts()

# Image manipulation

In [ ]:
x_train = []
y_train = []
target_shape = (IMG_HEIGHT, IMG_WIDTH)

def read_image(filepath, color=1, size=None):
    img = cv2.imread(filepath, color)
    if size:
        img = cv2.resize(img, size, interpolation = cv2.INTER_AREA)
    return img

def read_mask(directory, size=None):
    mask = np.zeros(image.shape[:1])
    for file in os.listdir(directory):
        mask_path = os.path.join(directory, file)
        mask_tmp = read_image(mask_path, color=0, size=target_shape)
        if not file: mask = mask_tmp
        else: mask = np.maximum(mask, mask_tmp)
    return mask

bright_field = np.array(lookup_table[lookup_table['type']=='bright-field']['img_id']) 
nissl_stained = np.array(lookup_table[lookup_table['type']=='nissl-stained']['img_id']) 
fluorescence = np.array(lookup_table[lookup_table['type']=='fluorescent']['img_id']) 

print('Resizing and inverting bright-field images, resizing masks ...')
sys.stdout.flush()

for i, filename in tqdm.tqdm(enumerate(bright_field), total=len(bright_field)):
    file_path = "input/stage1_train/{}/images/{}.png".format(filename, filename)
    image = read_image(file_path, size=target_shape)
    inv_image = skimage.util.invert(image)
    x_train.append(inv_image)
    for name in os.listdir("input/stage1_train/{}/masks/".format(filename)):
        masks_dir = "input/stage1_train/{}/masks".format(filename)
        masks = read_mask(masks_dir, size=target_shape)    
    y_train.append(masks)

print('Resizing, converting and inverting nissl_stained images, resizing masks ...')
sys.stdout.flush()
    
for i, filename in tqdm.tqdm(enumerate(nissl_stained), total=len(nissl_stained)):
    file_path = "input/stage1_train/{}/images/{}.png".format(filename, filename)
    image = read_image(file_path, size=target_shape, color=0)
    inv_image = skimage.util.invert(image)
    stacked_img = np.stack((inv_image,)*3, -1)
    x_train.append(stacked_img)
    for name in os.listdir("input/stage1_train/{}/masks/".format(filename)):
        masks_dir = "input/stage1_train/{}/masks".format(filename)
        masks = read_mask(masks_dir, size=target_shape)    
    y_train.append(masks)

print('Resizing, fluorescent images, resizing masks ...')
sys.stdout.flush()    
    
for i, filename in tqdm.tqdm(enumerate(fluorescence), total=len(fluorescence)):
    file_path = "input/stage1_train/{}/images/{}.png".format(filename, filename)
    image = read_image(file_path, size=target_shape)
    x_train.append(image)
    for name in os.listdir("input/stage1_train/{}/masks/".format(filename)):
        masks_dir = "input/stage1_train/{}/masks".format(filename)
        masks = read_mask(masks_dir, size=target_shape)    
    y_train.append(masks)


x_train = np.array(x_train)
y_train = np.expand_dims(np.array(y_train), axis=4)

print('These will be the model inputs')
print('x_train.shape: {} of dtype {}'.format(x_train.shape, x_train.dtype))
print('y_train.shape: {} of dtype {}'.format(y_train.shape, x_train.dtype))

In [ ]:
n=np.random.randint(len(x_train))
fig, axs = plt.subplots(1,2,figsize=(20,20))
axs[0].imshow(x_train[n])
axs[1].imshow(y_train[n,:,:,0], cm.gray)

In [ ]:
np.save('x_train', x_train)
np.save('y_train', y_train)


In [3]:
import numpy as np
x_train = np.load('x_train.npy')[:30]
x_train.shape

(30, 256, 256, 3)

In [5]:
import numpy as np
import keras
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Dense, Dropout, Flatten, Input
from keras.optimizers import SGD, Adam
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate, Add
import graphviz
from graphviz import Digraph
import os
os.environ['THEANO_FLAGS'] = "device=cuda,force_device=True,floatX=float32"
import theano


img_rows = 256
img_cols = 256
inputs = Input((img_rows, img_cols,3))
#x_train = np.random.random((100, 128, 128, 3))
#y_train = np.random.random((100, 128, 128, 1))
x_train = np.load('x_train.npy')[:30]
y_train = np.load('y_train.npy')[:30]

x_test = np.random.random((20, 100, 100, 3))
y_test = np.random.random((20, 100, 100, 1))

conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv11 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv11)


conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv22 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv22)


conv3 = Conv2D(256, 4, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv33 = Conv2D(256, 4, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv33)


conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv44 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = Dropout(0.5)(conv44)
pool4 = MaxPooling2D(pool_size=(2, 2),padding = 'same')(drop4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv55 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = Dropout(0.5)(conv55)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
merge6 = Add()([drop4,up6])
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = Add()([conv3,up7])
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = Add()([conv2,up8])
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = Add()([conv1,up9])
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = Model(inputs = inputs, outputs=conv10)
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(x_train, y_train, batch_size=1, epochs=1)

Epoch 1/1
30/30 [==============================] - 207s 7s/step - loss: 3.6299 - acc: 0.0349


## Verify score metrics for model

In [401]:
x_test = np.load('x_train.npy')[31:33]
y_test = np.load('y_train.npy')[31:33]

#y_pred = model.predict(np.expand_dims(x_test, axis=0)) #use this ONLY if your x_test has only one image
y_pred = model.predict(x_test)

In [402]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.segmentation

image = x_test
masks = y_test.squeeze()
num,height, width, _ = image.shape
num_masks = image.shape[0]

In [403]:
def get_labeled_mask(mask, cutoff=.5):
    """Object segmentation by labeling the mask."""
    mask = mask.reshape(mask.shape[0], mask.shape[1])
    lab_mask = skimage.morphology.label(mask > cutoff)
    (mask_labels, mask_sizes) = np.unique(lab_mask, return_counts=True)
    return lab_mask

This whole block should be part of a for loop, running through all the pairs of groud truth masks (which are the sub-arrays of `y_test[0]`, having shape 256,256) and the predicted masks (sub-arrays of `y_pred[0]`). It should start with something like:

`for i in len(y_pred[0]): true_mask = get_labeled_mask(masks[i]) ` etc.

In [427]:
true_mask = get_labeled_mask(masks[0])
pred_mask = get_labeled_mask(y_pred[0])

# Compute number of objects
true_objects = len(np.unique(true_mask))
pred_objects = len(np.unique(pred_mask))

# Compute intersection between all objects
intersection = np.histogram2d(true_mask.flatten(), pred_mask.flatten(), bins=(true_objects, pred_objects))[0]

# Compute areas (needed for finding the union between all objects)
area_true = np.histogram(labels, bins = true_objects)[0]
area_pred = np.histogram(y_pred, bins = pred_objects)[0]
area_true = np.expand_dims(area_true, -1)
area_pred = np.expand_dims(area_pred, 0)

# Compute union
union = area_true + area_pred - intersection

# Exclude background from the analysis
intersection = intersection[1:,1:]
union = union[1:,1:]
union[union == 0] = 1e-9

# Compute the intersection over union
iou = intersection / union

# Precision helper function
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1   # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
    p = tp / (tp + fp + fn)
    return tp, fp, fn, p

Then, instead of using `prec = []` you should build a pd.DataFrame or an np.array where you append the values obtained at every for loop of the previous block. The values then have to be averaged per each threshold.

In [428]:
# Loop over IoU thresholds
prec = []
for t in np.arange(0.5, 1.0, 0.05):
    tp, fp, fn, p = precision_at(t, iou)
    print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
    prec.append(p)
print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

0.500	0	0	24	0.000
0.550	0	0	24	0.000
0.600	0	0	24	0.000
0.650	0	0	24	0.000
0.700	0	0	24	0.000
0.750	0	0	24	0.000
0.800	0	0	24	0.000
0.850	0	0	24	0.000
0.900	0	0	24	0.000
0.950	0	0	24	0.000
AP	-	-	-	0.000


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

## Generate masks from test images 

In [71]:
import os
import cv2
import tqdm
IMG_WIDTH = 256     
IMG_HEIGHT = 256      

target_shape = (IMG_HEIGHT, IMG_WIDTH)

# WARNING! THIS STILL NEED TO BE EDITED TO IMPLEMENT  DIFFERENT TRANSFORMATIONS FOR EACH TYPE OF IMAGES

#test_list=os.listdir('input/stage1_test/')
for i, filename in tqdm.tqdm(enumerate(x_test[0])):
    #file_path = "input/stage1_test/{}/images/{}.png".format(filename, filename)
    #image = read_image(file_path, size=target_shape)
    x_test.append(image)
x_test = np.array(x_test)
y_file1 = model.predict(x_test)

65it [00:00, 131.49it/s]


(3, 256, 256, 1)

In [60]:
img